# Separating A Capella Songs into their Separated Vocal Tracks

One A Capella song is usually made up by 5-8 singers singing their respective parts together to form mainly the **lead_vocal, soprano, alto, bass, tenor, and vocal percussion.** <br>

In this notebook, we aim to **train our own Machine Learning Model** to **separate these 6 main tracks** from one another, given an A Capella song audio input. We will be using a dataset with Japanese A Capella songs (Ja Capella).

In [ ]:
!pip install librosa

In [ ]:
import glob
import math
import zipfile
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import random
import IPython.display as ipd



import scipy.signal as ss
import IPython.display as ipd


from glob import glob

In [ ]:
# Please ensure to have downloaded Ja Capella dataset and followed instructions from README.md before continuing

with zipfile.ZipFile("Dataset/Jacapella.zip", 'r') as zip_ref:
        zip_ref.extractall("Dataset/Jacapella")

In [ ]:
df = pd.read_csv('Dataset/Jacapella/meta.csv')

In [ ]:
df.sample(5)

In [ ]:
# create df_mix to show all the combines audio of mixed voice parts for each song
# append their file directory into the df_mix as well

titles = []
subset = []
file_dir = []
voice_part = []
for index in df.index:
    if df.iloc[index,0] not in titles:
        titles.append(df.iloc[index,0])
        subset.append(df.iloc[index,8])
        voice_part.append("mixture")
        file_dir.append(f"Dataset/Jacapella/{df.iloc[index,8]}/{df.iloc[index,0]}/mixture.wav")
        
df_mix = pd.DataFrame([], columns=['title_in_en', 'subset','voice_part','audio_file_dir'])
df_mix['title_in_en'] = titles
df_mix['subset'] = subset
df_mix['voice_part'] = voice_part
df_mix['audio_file_dir'] = file_dir

In [ ]:
df

In [ ]:
# Appending file directory of each vocal track
audio_file_dir = []
for index in df.index:
    title = df.iloc[index,0]
    subset = df.iloc[index,8]
    voice = df.iloc[index,9]
    audio_file_dir.append("Dataset/Jacapella/" + str(subset) + "/" + str(title) + "/" + str(voice) + ".wav")

# Added new column in df_audio to show each audio track's directory
df["audio_file_dir"] = audio_file_dir


In [ ]:
# creating separate df for each vocal parts

df_leadvocal = pd.DataFrame()
df_soprano = pd.DataFrame()
df_alto = pd.DataFrame()
df_tenor = pd.DataFrame()
df_bass = pd.DataFrame()
df_percussion = pd.DataFrame()

for index,value in df.iterrows():
    if df.iloc[index, 9] == "lead_vocal":
        df_leadvocal = df_leadvocal.append(value, ignore_index = True)
    elif df.iloc[index, 9] == "soprano":
        df_soprano = df_soprano.append(value, ignore_index = True)
    elif df.iloc[index, 9] == "alto":
        df_alto = df_alto.append(value, ignore_index = True)
    elif df.iloc[index, 9] == "tenor":
        df_tenor = df_tenor.append(value, ignore_index = True)
    elif df.iloc[index, 9] == "bass":
        df_bass = df_bass.append(value, ignore_index = True)
    else:
        df_percussion= df_percussion.append(value, ignore_index = True)

In [ ]:
df_leadvocal.drop(columns = ["title_in_ja", "lyric_writer", "copyright_of_lyric_writer", "composer", "copyright_of_composer", "arranger_in_en", "arranger_in_ja", "singer_id", "gender", "first_lang"], axis=1, inplace = True)
df_soprano.drop(columns = ["title_in_ja", "lyric_writer", "copyright_of_lyric_writer", "composer", "copyright_of_composer", "arranger_in_en", "arranger_in_ja", "singer_id", "gender", "first_lang"], axis=1, inplace = True)
df_alto.drop(columns = ["title_in_ja", "lyric_writer", "copyright_of_lyric_writer", "composer", "copyright_of_composer", "arranger_in_en", "arranger_in_ja", "singer_id", "gender", "first_lang"], axis=1, inplace = True)
df_tenor.drop(columns = ["title_in_ja", "lyric_writer", "copyright_of_lyric_writer", "composer", "copyright_of_composer", "arranger_in_en", "arranger_in_ja", "singer_id", "gender", "first_lang"], axis=1, inplace = True)
df_bass.drop(columns = ["title_in_ja", "lyric_writer", "copyright_of_lyric_writer", "composer", "copyright_of_composer", "arranger_in_en", "arranger_in_ja", "singer_id", "gender", "first_lang"], axis=1, inplace = True)
df_percussion.drop(columns = ["title_in_ja", "lyric_writer", "copyright_of_lyric_writer", "composer", "copyright_of_composer", "arranger_in_en", "arranger_in_ja", "singer_id", "gender", "first_lang"], axis=1, inplace = True)


# Listen to an example of Jacapella :D

In [ ]:
# Listen to an example of Jacapella :D
random.seed(0)
rand = random. randint(1,35)

print(f"Mixture audio for {df_mix.iloc[rand,0]}")
ipd.Audio(df_mix.iloc[rand,3])

In [ ]:
print(f"Soprano audio for {df_soprano.iloc[rand,0]}")
ipd.Audio(df_soprano.iloc[rand,3])

In [ ]:
print(f"Alto audio for {df_alto.iloc[rand,0]}")
ipd.Audio(df_alto.iloc[rand,3])

In [ ]:
print(f"Bass audio for {df_bass.iloc[rand,0]}")
ipd.Audio(df_bass.iloc[rand,3])

In [ ]:
print(f"Tenor audio for {df_tenor.iloc[rand,0]}")
ipd.Audio(df_tenor.iloc[rand,3])

In [ ]:
print(f"Percussion audio for {df_percussion.iloc[rand,0]}")
ipd.Audio(df_percussion.iloc[rand,3])

In [ ]:
print(f"Lead Vocal audio for {df_leadvocal.iloc[rand,0]}")
ipd.Audio(df_leadvocal.iloc[rand,3])

# Data Visualisation

In [ ]:
# Total of 7 genres, each has 5 songs.Each songs have 6 vocal parts.
df_mix['subset'].value_counts().plot(kind='bar', figsize=(10,3))
plt.xlabel("Genre")
plt.xticks(rotation=0)
plt.ylabel("Number of Samples")

**Amplitude Change for each vocal part**

The graph displays the time on the horizontal (X) axis and the amplitude on the vertical (Y) axis but it doesn’t tell us what’s happening to frequencies.

In [ ]:
# Add audio time series for each audio track

def add_time_series(df):
    
    # audio time series describes the amplitude of the audio at different timesteps.
    audio_time_series = []

    # sampling rate, sr
    sampling_rate = []

    for index, data in df.iterrows():
        y, sr = librosa.load(data['audio_file_dir'])
        audio_time_series.append(y)
        sampling_rate.append(sr)

    df["audio_time_series"] = audio_time_series
    df['sampling_rate'] = sampling_rate
    return df

In [ ]:
dfs = [df_leadvocal, df_soprano, df_alto, df_tenor, df_bass, df_percussion, df_mix]

In [ ]:
for df in dfs:
    add_time_series(df)

In [ ]:
# Visualise Amplitude Change which is the change of pressure near the microphone or recording device 
# for different vocal parts of 1 song

dfs = [df_leadvocal, df_soprano, df_alto, df_tenor, df_bass, df_percussion, df_mix]
random.seed(0)
rand = random. randint(1,35) 

def visualise_amp(df, index):
    row = df.iloc[index, :]
    pd.Series(row['audio_time_series']).plot(figsize=(10, 5), lw=1)
    plt.title(f"Amplitude change for {row['voice_part']} in {row['title_in_en']}")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.show()
    plt.tight_layout()

In [ ]:
for df in dfs:
    visualise_amp(df, rand)

# Feature Engineering

**1. Zero Crossing Rate (ZCR)**

The zero-crossing rate describes the rate at which a signal moves from positive to zero to negative or from negative to zero to positive. The feature is used in music retrieval systems to identify noisy signals.

ZCR is a feature often used in signal processing and audio analysis. Change in ZCR could be due to transitioning from silence to sound, or change in pitch, or change in environmental sound.

In [ ]:
# Concat all dfs tgt 

df_all = pd.concat([df_leadvocal, df_soprano, df_alto, df_tenor, df_bass, df_percussion, df_mix], axis=0, ignore_index = True)
df_all['voice_part'].nunique()

df_all.sample(5)

In [ ]:
def zcr(df):
    ZCR = []
    for index, song in df.iterrows():
        zero_crossings = librosa.zero_crossings(song['audio_time_series'],pad=False)
        ZCR.append(sum(zero_crossings))

    df["zcr"] = ZCR
    return df

zcr(df_all)

In [ ]:
# plot 2-D btw voice part & ZCR
#df_audio[['subset', 'zcr']].plot(kind='scatter', x='subset', y='zcr')

sns.scatterplot(x='subset', y='zcr', hue='voice_part', data=df_all)
sns.set(rc = {'figure.figsize':(10, 10)})
plt.show()

In [ ]:
# Add zcr into each individual dfs
for df in dfs:
    zcr(df)


**2. Spectral Centroid** <br>

It indicates where the ”centre of mass” for a sound is located and is calculated as the weighted mean of the frequencies present in the sound. 

In [ ]:
#spectral centroid -- centre of mass -- weighted mean of the frequencies present in the sound
import sklearn

def spectral_centr(df):
    num_frames = []
    centroids = []
    for index, data in df.iterrows():
        x = data['audio_time_series']
        spectral_centroids = librosa.feature.spectral_centroid(y=x, sr=data['sampling_rate'])
#         print(spectral_centroids.shape[1])
#         print(spectral_centroids[0])
        num_frames.append(spectral_centroids.shape[1])

        # Computing the time variable for visualization
        frames = range(len(spectral_centroids[0]))
        t = librosa.frames_to_time(frames)

        centroids.append([spectral_centroids[0],t])
        
    df['num_frames'] = num_frames
    df['spectral_centroid'] = centroids
        
    return df


In [ ]:
for df in dfs:
    spectral_centr(df)


**3. MFCC — Mel-Frequency Cepstral Coefficients** <br>

MFCCs of a signal are a small set of features (usually about 10–20) which concisely describe the overall shape of a spectral envelope. 

MFCC is used for deduction of noise in audios and also used for audio classification. They represent the audio's spectral characteristics and are commonly used in audio processing tasks such as music information retrieval and speech recognition.

In [ ]:
def mfcc(df):
    mfcc_coefficient = []
    for index, data in df.iterrows():
        mfcc = librosa.feature.mfcc(y= data['audio_time_series'], sr= data['sampling_rate'])
        mfcc_coefficient.append(mfcc)
        
    df['mfcc'] = mfcc_coefficient
    return df

In [ ]:
for df in dfs:
    mfcc(df)

**4. Short-Term Fourier Transform (STFT)** <br>

Its complex-valued coefficients provide the frequency and phase content of local sections of a signal as it evolves over time.

In [ ]:
def stft(df):
    stft = []
    for index, data in df.iterrows():
        
        # Return the complex Short Term Fourier Transform
        y = data['audio_time_series']
        sound_stft = np.abs(librosa.stft(y))
        stft.append(sound_stft)
                                       
    df['stft'] = stft
    return df

In [ ]:
for df in dfs:
    stft(df)
    print( stft(df))

**Magnitude Spectrogram**

A spectrogram is a detailed view of a signal that covers all three characteristics of sound.<br> 
X-axis represents time, Y-axis represent frequency, color represents amplitude. 

The louder the event the brighter the color, while silence is represented by black.

In [ ]:
# Magnitude spectrogram that visualises the 3 aspects of STFT

random.seed(0)
rand = random. randint(1,35) 

def spectrogram(df, index):
    row = df.iloc[index, :]
#     X = librosa.stft(row['audio_time_series'])
    Xdb = librosa.amplitude_to_db(abs(row['stft']))
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(Xdb, sr=row['sampling_rate'], x_axis='time', y_axis='hz') 
    plt.title(f"Spectrogram for {row['voice_part']} in {row['title_in_en']}")
    plt.colorbar()

In [ ]:
for df in dfs:
    spectrogram(df, rand)

**5. Pitch Features**

In [ ]:
# EDA on pitch features
# Lists to store pitch analysis results

def extract_pitch_and_analyze(df):
    mean_pitch_list = []
    median_pitch_list = []
    min_pitch_list = []
    max_pitch_list = []
#     pitch_compile = []
    
    # Function to extract pitch and perform analysis
    for index, data in df.iterrows():
        # Extract pitch features using the YIN algorithm
        pitch, magnitudes = librosa.piptrack(y=data['audio_time_series'], sr=data['sampling_rate'])
        pitch = pitch[pitch > 0]  # Filter out non-positive pitch values
#         pitch_compile.append(pitch)
        
        # Calculate pitch statistics
        mean_pitch = pitch.mean()
        median_pitch = np.median(pitch)
        min_pitch = pitch.min()
        max_pitch = pitch.max()

        # Append pitch statistics to the respective lists
        mean_pitch_list.append(mean_pitch)
        median_pitch_list.append(median_pitch)
        min_pitch_list.append(min_pitch)
        max_pitch_list.append(max_pitch)
    
#     df['pitch'] = pitch
    df['mean_pitch'] = mean_pitch_list
    df['median_pitch'] = median_pitch_list
    df['min_pitch'] = min_pitch_list
    df['max_pitch'] = max_pitch_list

In [ ]:
# calc mean pitch for individual vocal parts and put into their own dataframes.
for df in dfs:
    extract_pitch_and_analyze(df)
    

In [ ]:
# combine mean pitch for all vocal parts tgt to show histogram
extract_pitch_and_analyze(df_all)

In [ ]:
# histogram with hue based on 'voice_part' for mean pitch
plt.figure(figsize=(10, 6))
sns.histplot(data=df_all, x='mean_pitch', hue='voice_part', element='step', bins=20, kde=True, hue_order=df_all['voice_part'].unique())
plt.title("Histogram of Mean Pitch by Voice Part")
plt.xlabel("Mean Pitch")
plt.ylabel("Frequency")
plt.legend(title="Voice Part", labels=df_all['voice_part'].unique())
plt.figure(figsize=(10, 6))
plt.show()

**6. Chroma feature extraction** <br>

Chroma feature extraction is a technique commonly used in music signal processing to represent the harmonic content of an audio signal. <br> It aims to capture the distribution of pitch classes, which are the 12 distinct notes in the Western music scale (C, C#, D, D#, E, F, F#, G, G#, A, A#, B).

In [ ]:
def chroma(df):
    chroma_feature = []
    for index, data in df.iterrows():
        # Compute chroma feature
        chroma = librosa.feature.chroma_stft(y=data['audio_time_series'], sr=data['sampling_rate'])
        chroma_feature.append(chroma)
    df['chroma'] = chroma_feature

In [ ]:
for df in dfs:
    chroma(df)

In [ ]:
random.seed(0)
rand = random. randint(1,35) 

def visualise_chroma(df, index):
    # Get the randomly selected song title row's data
    row = df.iloc[index,:]
    
    # Visualize the Chroma feature matrix for diff parts of that same song
    plt.figure(figsize=(8, 4))
    librosa.display.specshow(row['chroma'], y_axis='chroma', x_axis='time')
    plt.colorbar()
    plt.title(f"Chroma Feature for {row['voice_part']} in {row['title_in_en']} ")
    plt.xlabel("Time (s)")
    plt.ylabel("Chroma")
    plt.show()


In [ ]:
for df in dfs:
    visualise_chroma(df, rand)

# Non-negative matrix factorization (NMF) 

NMF is a powerful sound source separation technique that can extract individual sound sources from a mixture of sounds.

#### EDA

In [ ]:
###initial signal waveform

df_mix.iloc[rand,0]

# Plotting the sound's signal waveform
fig, ax = plt.subplots(figsize=(10, 3))
librosa.display.waveshow(df_mix.iloc[0,4], sr=22050, ax=ax,x_axis='time')
ax.set(title='The sound waveform', xlabel='Time [s]')
ax.legend()

In [ ]:
stft_1 = df['stft'][0]
stft_1

In [ ]:
example_Xdb = librosa.amplitude_to_db(abs(stft_1))
plt.figure(figsize=(10, 5))
librosa.display.specshow(example_Xdb, sr=22050, x_axis='time', y_axis='hz') 


plt.title(f"Spectrogram ")
plt.colorbar()

### NMF Attempt

In [ ]:
#NMF Method 1: Just trying  NMF seperation

#1 seperate
from sklearn.decomposition import NMF

n_components = 2  # Number of components, adjust as needed
nmf = NMF(n_components=n_components, init='nndsvd', max_iter=200)
W = nmf.fit_transform(abs(stft_1))
H = nmf.components_

In [ ]:
# Visualize the basis matrix (W)
plt.figure(figsize=(8, 4))
librosa.display.specshow(librosa.amplitude_to_db(W.T), x_axis='time', y_axis='log')
plt.colorbar(format='%+5.0f dB')
plt.title('Basis Matrix (W)')
plt.show()

In [ ]:
# Visualize the activation matrix (H)
plt.figure(figsize=(8, 4))
librosa.display.specshow(H, x_axis='time')
plt.colorbar()
plt.title('Activation Matrix (H)')
plt.show()

In [ ]:
#2 reconstruct 

# Select the basis vector and activation coefficient for the vocals
vocals_basis_vector = W[:, 1]  # Adjust the index as needed
vocals_activation_coefficient = H[1, :]

# Reconstruct the separated vocals
separated_vocals = vocals_basis_vector[:, np.newaxis] @ vocals_activation_coefficient[np.newaxis, :]

In [ ]:
#3 invert
separated_audio = librosa.istft(separated_vocals)

In [ ]:
#4 save seperate vocals
import soundfile as sf
sf.write('attempt_1.wav', separated_audio, 22050)

In [ ]:
##Method 2: try optimized seperation using a cost function
#https://medium.com/@zahrahafida.benslimane/audio-source-separation-using-non-negative-matrix-factorization-nmf-a8b204490c7d

In [ ]:
def divergence(V,W,H, beta = 2):
    
    """
    beta = 2 : Euclidean cost function
    beta = 1 : Kullback-Leibler cost function
    beta = 0 : Itakura-Saito cost function
    """ 
    
    if beta == 0 : return np.sum( V/(W@H) - math.log10(V/(W@H)) -1 )
    
    if beta == 1 : return np.sum( V*math.log10(V/(W@H)) + (W@H - V))
    
    if beta == 2 : return 1/2*np.linalg.norm(W@H-V)

In [ ]:
def NMF(V, S, beta = 2,  threshold = 0.05, MAXITER = 5000): 
    
    """
    inputs : 
    --------
        V         : Mixture signal : |TFST|
        S         : The number of sources to extract
        beta      : Beta divergence considered, default=2 (Euclidean)
        threshold : Stop criterion 
        MAXITER   : The number of maximum iterations, default=1000                                                     
    
    outputs :
    ---------
        W : dictionary matrix [KxS], W>=0
        H : activation matrix [SxN], H>=0
        cost_function : the optimised cost function over iterations
       
   Algorithm : 
   -----------
   
    1) Randomly initialize W and H matrices
    2) Multiplicative update of W and H 
    3) Repeat step (2) until convergence or after MAXITER   
    """
    
    counter = 0
    cost_function = []
    beta_divergence = 1
    
    K, N = np.shape(V)
    
    # Initialisation of W and H matrices : The initialization is generally random
    W = np.abs(np.random.normal(loc=0, scale = 2.5, size=(K,S)))    
    H = np.abs(np.random.normal(loc=0, scale = 2.5, size=(S,N)))

    while beta_divergence >= threshold and counter <= MAXITER:
        
        # Update of W and H
        H *= (W.T@(((W@H)**(beta-2))*V))/(W.T@((W@H)**(beta-1)) + 10e-10)
        W *= (((W@H)**(beta-2)*V)@H.T)/((W@H)**(beta-1)@H.T + 10e-10)
        
        # Compute cost function
        beta_divergence =  divergence(V,W,H, beta = 2)
        cost_function.append( beta_divergence )
        counter += 1
       
    return W,H, cost_function

In [ ]:
#prepare magnitude spectrogram: np.abs(stft_1)

magnitude_spectrogram = np.abs(stft_1)
phase_spectrogram = np.angle(stft_1)

In [ ]:
V = magnitude_spectrogram + 1e-10
beta = 2
S = 3 

# Applying the NMF function
W, H, cost_function = NMF(V,S,beta = beta, threshold = 0.05, MAXITER = 5000) 

# Ploting the cost function
plt.figure(figsize=(5,3))
plt.plot(cost_function)
plt.title("Cost Function")
plt.xlabel("Number of iteration")
plt.ylabel(f"Beta Divergence for beta = {beta} ")

In [ ]:
#Visualization of audio sources

In [ ]:
#After NMF, each audio source S can be expressed as a frequency mask over time
f, axs = plt.subplots(nrows=1, ncols=S,figsize=(20,5))
filtered_spectrograms = []
for i in range(S):
    axs[i].set_title(f"Frequency Mask of Audio Source s = {i+1}") 
    # Filter eash source components
    filtered_spectrogram = W[:,[i]]@H[[i],:]
    # Compute the filtered spectrogram
    D = librosa.amplitude_to_db(filtered_spectrogram, ref = np.max)
    # Show the filtered spectrogram
    librosa.display.specshow(D,y_axis = 'hz', sr=22050,hop_length=256,x_axis ='time',cmap= matplotlib.cm.jet, ax = axs[i])
    
    filtered_spectrograms.append(filtered_spectrogram)

In [ ]:
#reconstruct

In [ ]:
reconstructed_sounds = []
for i in range(S):
    reconstruct = filtered_spectrograms[i] * np.exp(1j*phase_spectrogram)
    new_sound   = librosa.istft(reconstruct, hop_length = 256)
    reconstructed_sounds.append(new_sound)

In [ ]:
reconstructed_sounds

In [ ]:
# Tracing the waveform
colors = ['r', 'g','b']
fig, ax = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(10, 8))
for i in range(S):
    librosa.display.waveshow(reconstructed_sounds[i], sr=22050, color = colors[i], ax=ax[i],label=f'Source {i}',x_axis='time')
    ax[i].set(xlabel='Time [s]')
    ax[i].legend()

In [ ]:
import soundfile as sf

sf.write('separated_vocals_exp_source0.wav', reconstructed_sounds[0], 22050)
sf.write('separated_vocals_exp_source1.wav', reconstructed_sounds[1], 22050)
sf.write('separated_vocals_exp_source2.wav', reconstructed_sounds[2], 22050)

In [ ]:
## Attempt 3: Trying Blind Source Seperation  
#https://sound-source-separation-python.readthedocs.io/en/v0.1.7/

In [ ]:
final_path = df_mix.iloc[rand,3]
final_path

In [ ]:
from scipy.io import wavfile



# Read the .wav files of singular sauces
sample_rate_s, soprano_wave = wavfile.read(df_soprano.iloc[rand,3])
sample_rate_a, alto_wave = wavfile.read(df_alto.iloc[rand,3])
sample_rate_t, tenor_wave = wavfile.read(df_tenor.iloc[rand,3])
sample_rate_b, bass_wave = wavfile.read(df_bass.iloc[rand,3])
sample_rate_p, percussion_wave = wavfile.read(df_percussion.iloc[rand,3])
sample_rate_lv, lead_vocal_wave = wavfile.read(df_leadvocal.iloc[rand,3])

In [ ]:
print(f"Sample rate (soprano): {sample_rate_s}")
print(f"Waveform (soprano): {soprano_wave}")
soprano_wave.shape

In [ ]:
import matplotlib.pyplot as plt

# Assuming soprano_wave is your audio waveform
plt.figure(figsize=(12, 4))
plt.plot(soprano_wave)
plt.title('Soprano Audio Waveform')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.show()


#indiv waveforms are working

In [ ]:
soprano_stereo_waveform = np.stack((soprano_wave, soprano_wave, soprano_wave), axis=0)
alto_stereo_waveform = np.stack((alto_wave, alto_wave, alto_wave), axis=0)
tenor_stereo_waveform = np.stack((tenor_wave, tenor_wave, tenor_wave), axis=0)
bass_stereo_waveform = np.stack((bass_wave, bass_wave, bass_wave), axis=0)
percussion_stereo_waveform = np.stack((percussion_wave, percussion_wave, percussion_wave), axis=0)
lead_vocal_stereo_waveform = np.stack((lead_vocal_wave, lead_vocal_wave, lead_vocal_wave), axis=0)

print(soprano_stereo_waveform)
soprano_stereo_waveform.shape

In [ ]:
tenor_stereo_waveform.shape


In [ ]:
waveforms = []
waveforms.append(soprano_wave)
waveforms.append(alto_wave) 
waveforms.append(tenor_wave) 
waveforms.append(bass_wave) 
waveforms.append(percussion_wave) 
waveforms.append(lead_vocal_wave) #combine all separated audio together

waveforms

In [ ]:
waveforms_stereo= []

waveforms_stereo.append(soprano_stereo_waveform)
waveforms_stereo.append(alto_stereo_waveform) 
waveforms_stereo.append(tenor_stereo_waveform) 
waveforms_stereo.append(bass_stereo_waveform) 
waveforms_stereo.append(percussion_stereo_waveform) 
waveforms_stereo.append(lead_vocal_stereo_waveform) #combine all separated audio together

waveforms_stereo

In [ ]:
waveforms = np.stack(waveforms, axis=1)
print(waveforms[0:6])

In [ ]:
waveforms_stereo = np.stack(waveforms_stereo, axis=1)
print(waveforms_stereo[0:6])
waveforms_stereo[0:6].shape

In [ ]:
waveforms_concat = soprano_wave + alto_wave + tenor_wave + bass_wave + percussion_wave + lead_vocal_wave
waveforms_concat

In [ ]:
import matplotlib.pyplot as plt

# Assuming soprano_wave is your audio waveform
plt.figure(figsize=(12, 4))
plt.plot(waveforms_concat)
plt.title('Mixed Audio Waveform')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.show()


#indiv waveforms are stacked

In [ ]:
print("Mixture of waveforms: {}")
display(ipd.Audio(waveforms_stereo_concat, rate=sample_rate_s))

In [ ]:
#reshaping to get w_img form 

# Reshape the 1D array into a 3D array
desired_shape = (2, 2, -1)  # Define the desired shape (2 planes, 2 rows, and -1 for automatic columns)  
#how do i get desired shape without screwing up the waveform , duplicate into shape?

reshaped_wave = waveforms_concat.reshape(desired_shape)

print(reshaped_wave)

In [ ]:
#get w_mix 

waveform_mix = np.sum(reshaped_wave, axis=1) 

In [ ]:
waveform_mix.shape

In [ ]:
waveform_mix

In [ ]:
#slice wave 

# Calculate the new size (1/10th of the original size)
new_size = waveform_mix.shape[1] // 10

# Calculate the starting index to get the middle portion
start_index = (waveform_mix.shape[1] - new_size) // 2

# Calculate the ending index
end_index = start_index + new_size

# Slice the waveform to get the middle portion
waveform_sample_mix = waveform_mix[:, start_index:end_index]

In [ ]:
waveform_sample_mix.shape

In [ ]:
#part 2: using blind source seperation model

from ssspy.bss.ilrma import GaussILRMA


In [ ]:
ilrma = GaussILRMA(n_basis=3, rng=np.random.default_rng(0))  # 3 basis matrices
print(ilrma)

In [ ]:
n_fft, hop_length = 2048, 512


In [ ]:
import psutil

# Get the available physical memory in bytes
available_memory = psutil.virtual_memory().available

# Convert the available memory to a more human-readable format
available_memory_gb = available_memory / (1024**3)  # Convert bytes to gigabytes

print(f"Available Memory: {available_memory_gb:.2f} GB")

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_sample_mix, window = "hann")
#, window=window, nperseg=n_fft, noverlap=n_fft-hop_length

In [ ]:
waveform_sample_mix.shape

In [ ]:
spectrogram_est = ilrma(spectrogram_mix, n_iter=500)


#The shape is (n_channels, n_bins, n_frames)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est,window = "hann")  # window=window, nperseg=n_fft, noverlap=n_fft-hop_length


In [ ]:
waveform_est

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate_s))
    print()    
    

In [ ]:
import matplotlib.pyplot as plt
import librosa


# Load your sound waveform
sound_waveform, sound_sample_rate = waveform_est[0],sample_rate_s

# Load your ground truth waveform
ground_truth_waveform, gt_sample_rate = librosa.load(df_mix.iloc[0,3])


In [ ]:
# Check if they have the same length
if ground_truth_waveform.shape[0] != sound_waveform.shape[0]:
    # Trim the longer waveform to match the length of the shorter waveform
    min_length = min(ground_truth_waveform.shape[0], sound_waveform.shape[0])
    
    reference_sources = ground_truth_waveform[:min_length]
    estimated_sources = sound_waveform[:min_length]

In [ ]:
# Create a time vector for the x-axis
sound_duration = len(estimated_sources) / sound_sample_rate
sound_time = librosa.times_like(estimated_sources, sr=sound_sample_rate)

# Plot the sound waveform
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.title('Sound Waveform')
plt.plot(sound_time, estimated_sources, color='b')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Plot the ground truth waveform
plt.subplot(2, 1, 2)
plt.title('Ground Truth Waveform')
gt_time = librosa.times_like(reference_sources, sr=gt_sample_rate)
plt.plot(gt_time, reference_sources, color='r')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

In [ ]:
estimated_sources.shape

In [ ]:
import mir_eval

# Calculate SDR
sdr, sir, sar, _ = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)

print(sdr, sir, sar,)

In [ ]:
plt.figure()
plt.plot(range(1, len(ilrma.loss)), ilrma.loss[1:])
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()
plt.close()